In [1]:
from tensorflow.examples.tutorials.mnist import input_data





C:\Users\Phansa Chaonpoj\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
import sys
import numpy as np
import pickle

from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

#from MNIST_Dataset_Loader.mnist_loader import MNIST



import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')






old_stdout = sys.stdout
log_file = open("summary.log","w")
sys.stdout = log_file


print('\nLoading MNIST Data...')

In [26]:
class A():
    def __init__(self):
        print('\nLoading Training Data...')
        img_train, labels_train = MNIST.load_training(self)
        train_img = np.array(img_train)
        train_labels = np.array(labels_train)

        print('\nLoading Testing Data...')
        img_test, labels_test = MNIST.load_testing(self)
        test_img = np.array(img_test)
        test_labels = np.array(labels_test)


        #Features
        X = train_img
        print(X)

        #Labels
        y = train_labels

        print('\nPreparing Classifier Training and Validation Data...')
        X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y,test_size=0.1)


        print('\nKNN Classifier with n_neighbors = 5, algorithm = auto, n_jobs = 10')
        print('\nPickling the Classifier for Future Use...')
        clf = KNeighborsClassifier(n_neighbors=5,algorithm='auto',n_jobs=10)
        clf.fit(X_train,y_train)

        with open('MNIST_KNN.pickle','wb') as f:
            pickle.dump(clf, f)

        pickle_in = open('MNIST_KNN.pickle','rb')
        clf = pickle.load(pickle_in)

        print('\nCalculating Accuracy of trained Classifier...')
        confidence = clf.score(X_test,y_test)

        print('\nMaking Predictions on Validation Data...')
        y_pred = clf.predict(X_test)

        print('\nCalculating Accuracy of Predictions...')
        accuracy = accuracy_score(y_test, y_pred)

        print('\nCreating Confusion Matrix...')
        conf_mat = confusion_matrix(y_test,y_pred)

        print('\nKNN Trained Classifier Confidence: ',confidence)
        print('\nPredicted Values: ',y_pred)
        print('\nAccuracy of Classifier on Validation Image Data: ',accuracy)
        print('\nConfusion Matrix: \n',conf_mat)


        # Plot Confusion Matrix Data as a Matrix
        plt.matshow(conf_mat)
        plt.title('Confusion Matrix for Validation Data')
        plt.colorbar()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        plt.show()


        print('\nMaking Predictions on Test Input Images...')
        test_labels_pred = clf.predict(test_img)

        print('\nCalculating Accuracy of Trained Classifier on Test Data... ')
        acc = accuracy_score(test_labels,test_labels_pred)

        print('\n Creating Confusion Matrix for Test Data...')
        conf_mat_test = confusion_matrix(test_labels,test_labels_pred)

        print('\nPredicted Labels for Test Images: ',test_labels_pred)
        print('\nAccuracy of Classifier on Test Images: ',acc)
        print('\nConfusion Matrix for Test Data: \n',conf_mat_test)

        # Plot Confusion Matrix for Test Data
        plt.matshow(conf_mat_test)
        plt.title('Confusion Matrix for Test Data')
        plt.colorbar()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        plt.axis('off')
        plt.show()

        sys.stdout = old_stdout
        log_file.close()



# Show the Test Images with Original and Predicted Labels
        a = np.random.randint(1,50,20)
        for i in a:
            two_d = (np.reshape(test_img[i], (28, 28)) * 255).astype(np.uint8)
            plt.title('Original Label: {0}  Predicted Label: {1}'.format(test_labels[i],test_labels_pred[i]))
            plt.imshow(two_d, interpolation='nearest',cmap='gray')
            plt.show()

In [6]:
from MNIST_Dataset_Loader.mnist_loader import MNIST

In [8]:
import os
import struct
from mnist import MNIST
from array import array


class MNIST(object):
    def __init__(self, path='.'):
        self.path = path

        self.test_img_fname = '.t10k-images-idx3-ubyte'
        self.test_lbl_fname = '.t10k-labels-idx1-ubyte'

        self.train_img_fname = '.train-images-idx3-ubyte'
        self.train_lbl_fname = '.train-labels-idx1-ubyte'

        self.test_images = []
        self.test_labels = []

        self.train_images = []
        self.train_labels = []

    def load_testing(self):
        ims, labels = self.load(os.path.join(self.path, self.test_img_fname),
                                os.path.join(self.path, self.test_lbl_fname))

        self.test_images = ims
        self.test_labels = labels

        return ims, labels

    def load_training(self):
        ims, labels = self.load(os.path.join(self.path, self.train_img_fname),
                                os.path.join(self.path, self.train_lbl_fname))

        self.train_images = ims
        self.train_labels = labels

        return ims, labels

    @classmethod
    def load(cls, path_img, path_lbl):
        with open(path_lbl, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049,'
                                 'got {}'.format(magic))

            labels = array("B", file.read())

        with open(path_img, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051,'
                                 'got {}'.format(magic))

            image_data = array("B", file.read())

        images = []
        for i in range(size):
            images.append([0] * rows * cols)

        for i in range(size):
            images[i][:] = image_data[i * rows * cols:(i + 1) * rows * cols]

        return images, labels

    @classmethod
    def display(cls, img, width=28, threshold=200):
        render = ''
        for i in range(len(img)):
            if i % width == 0:
                render += '\n'
            if img[i] > threshold:
                render += '@'
            else:
                render += '.'
        return render


In [24]:

A()

AttributeError: 'A' object has no attribute 'load'

NameError: name 'X' is not defined